# Step 4: Get stock historical data and compute the 14 indicators

## 1. Import required libraries

In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime
import numpy as np

## 2. List of tickers to get data for

In [2]:
tick_list =  ['AAPL', 'AMC', 'DKNG', 'TSLA', 'AMD', 'BABA']

## 3. Get ticker data and save

### 3.1 Enter start and end dates

In [3]:
#We keep an early start date from what is required because the parameter calculations have look back periods of upto 34 days
start = datetime(2020,6,1) 
end = datetime(2021,9,30) 

### 3.2 Get price data for each ticker, compute technical indicators and save the data in a pickle file

In [5]:
for tick in tick_list:
    pltr = yf.Ticker(tick) 
    pltr_history = pltr.history(start=start, end=end)
    pltr_history['YearMonDay'] = pltr_history.apply(lambda row: datetime.fromisoformat(str(row.name)).strftime("%y%b%d"), axis=1)
    pltr_history.reset_index(inplace=True)
    pltr_history['Close-t'] = pltr_history['Close'].rolling(min_periods=2, window=2).apply(lambda x: x.iloc[0])
    pltr_history['log_ret'] = np.log(pltr_history.Close) - np.log(pltr_history.Close.shift(1))
    pltr_history['log_ret-t'] = pltr_history['log_ret'].rolling(min_periods=2, window=2).apply(lambda x: x.iloc[0])    
    
    ###-----ACD
    pltr_history['N_multiplier'] = pltr_history.Volume*((pltr_history.Close - pltr_history.Low) - (pltr_history.High - pltr_history.Close))/(pltr_history.High - pltr_history.Low)
    pltr_history['ACD'] = pltr_history['N_multiplier'].rolling(min_periods=2, window=2).sum()
    pltr_history.drop('N_multiplier', axis=1, inplace=True)

    ###-----MACD
    pltr_history['EMA_Close_12'] = pltr_history.loc[:,'Close'].ewm(span=12, min_periods=12, adjust=False).mean()
    pltr_history['EMA_Close_26'] = pltr_history.loc[:,'Close'].ewm(span=26, min_periods=26, adjust=False).mean()
    pltr_history['MACD'] = pltr_history['EMA_Close_12'] - pltr_history['EMA_Close_26']
    pltr_history.drop('EMA_Close_12', axis=1, inplace=True)
    pltr_history.drop('EMA_Close_26', axis=1, inplace=True)

    ###-----CHO
    pltr_history['EMA_ACD_3'] = pltr_history.loc[:,'ACD'].ewm(span=3, min_periods=2,adjust=False).mean()
    pltr_history['EMA_ACD_10'] = pltr_history.loc[:,'ACD'].ewm(span=10, min_periods=10,adjust=False).mean()
    pltr_history['CHO'] = pltr_history['EMA_ACD_3'] - pltr_history['EMA_ACD_10']
    pltr_history.drop('EMA_ACD_3', axis=1, inplace=True)
    pltr_history.drop('EMA_ACD_10', axis=1, inplace=True)

    ###-----Highest T
    pltr_history['Highest_T'] = pltr_history['Close'].rolling(min_periods=14, window=14).max()

    ###-----Lowest T
    pltr_history['Lowest_T'] = pltr_history['Close'].rolling(min_periods=14, window=14).min()

    ###-----S-K
    pltr_history['Highest_5'] = pltr_history['Close'].rolling(min_periods=5, window=5).max()
    pltr_history['Lowest_5'] = pltr_history['Close'].rolling(min_periods=5, window=5).min()
    pltr_history['S-K'] = ((pltr_history['Close'] - pltr_history['Lowest_5'])/(pltr_history['Highest_5'] - pltr_history['Lowest_5'])*100)
    pltr_history.drop('Highest_5', axis=1, inplace=True)
    pltr_history.drop('Lowest_5', axis=1, inplace=True)

    ###-----S-D
    pltr_history['S-D'] = pltr_history['S-K'].rolling(min_periods=3, window=3).mean()

    ###-----VPT
    pltr_history['VPT'] = 0
    for i in range(len(pltr_history)): 
        if i==0 or i==1:
            pltr_history.loc[i, 'VPT'] = 0
        else:   
            vpt_t = ((pltr_history.loc[i, 'Close'] - pltr_history.loc[i-1, 'Close'])/pltr_history.loc[i-1, 'Close'])*pltr_history.loc[i, 'Volume']
            vpt_t1 = ((pltr_history.loc[i-1, 'Close'] - pltr_history.loc[i-2, 'Close'])/pltr_history.loc[i-2, 'Close'])*pltr_history.loc[i-1, 'Volume']
            pltr_history.loc[i, 'VPT'] = vpt_t + vpt_t1

    ###-----W-R
    pltr_history['Highest_14'] = pltr_history['Close'].rolling(min_periods=1, window=14).max()
    pltr_history['Lowest_14'] = pltr_history['Close'].rolling(min_periods=1, window=14).min()       
    pltr_history['W-R'] = ((pltr_history['Highest_14']-pltr_history['Close'])/(pltr_history['Highest_14']-pltr_history['Lowest_14'])*100)
    pltr_history.drop('Highest_14', axis=1, inplace=True)
    pltr_history.drop('Lowest_14', axis=1, inplace=True)

    ###-----RSI
    pltr_history['Up_Move'] = pltr_history['Close'].rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0] if (x.iloc[1] - x.iloc[0]) > 0 else 0 )
    pltr_history['Down_Move'] = pltr_history['Close'].rolling(window=2).apply(lambda x: abs(x.iloc[1] - x.iloc[0]) if (x.iloc[1] - x.iloc[0]) < 0 else 0 )
    pltr_history['Avg_Up_14'] = pltr_history['Up_Move'].rolling(min_periods=14, window=14).mean()
    pltr_history['Avg_Down_14'] = pltr_history['Down_Move'].rolling(min_periods=14, window=14).mean()
    pltr_history['RSI'] = 100 - (100/(1+(pltr_history['Avg_Up_14']/pltr_history['Avg_Down_14'])))
    pltr_history.drop('Up_Move', axis=1, inplace=True)
    pltr_history.drop('Down_Move', axis=1, inplace=True)
    pltr_history.drop('Avg_Up_14', axis=1, inplace=True)
    pltr_history.drop('Avg_Down_14', axis=1, inplace=True)

    ###-----MOME
    pltr_history['MOME'] = pltr_history['Close'].rolling(min_periods=14, window=14).apply(lambda x: x.iloc[13] - x.iloc[0])

    ###-----AC
    pltr_history['Median_Price'] = (pltr_history['High'] + pltr_history['Low'])/2
    pltr_history['Median_MA_5'] = pltr_history['Median_Price'].rolling(min_periods=5, window=5).mean()
    pltr_history['Median_MA_34'] = pltr_history['Median_Price'].rolling(min_periods=34, window=34).mean()
    pltr_history['AO'] = pltr_history['Median_MA_5'] - pltr_history['Median_MA_34']
    pltr_history['AO_MA_5'] = pltr_history['AO'].rolling(min_periods=5, window=5).mean()
    pltr_history['AC'] = pltr_history['AO'] - pltr_history['AO_MA_5']
    pltr_history.drop('Median_Price', axis=1, inplace=True)
    pltr_history.drop('Median_MA_5', axis=1, inplace=True)
    pltr_history.drop('Median_MA_34', axis=1, inplace=True)
    pltr_history.drop('AO', axis=1, inplace=True)
    pltr_history.drop('AO_MA_5', axis=1, inplace=True)

    ###-----#PROC
    pltr_history['PROC'] = pltr_history['Close'].rolling(min_periods=12, window=12).apply(lambda x: ((x.iloc[11] - x.iloc[0])/x.iloc[0])*100)

    ###-----VROC
    pltr_history['VROC'] = pltr_history['Volume'].rolling(min_periods=12, window=12).apply(lambda x: ((x.iloc[11] - x.iloc[0])/x.iloc[0])*100)

    ###-----OBV
    pltr_history['OBV'] = 0
    for i in range(len(pltr_history)): 
        if i==0:
            continue
        else:   
            if pltr_history.loc[i, 'Close'] - pltr_history.loc[i-1, 'Close'] > 0:
                pltr_history.loc[i, 'OBV'] = pltr_history.loc[i-1, 'OBV'] + pltr_history.loc[i, 'Volume']
            elif pltr_history.loc[i, 'Close'] - pltr_history.loc[i-1, 'Close'] < 0:
                pltr_history.loc[i, 'OBV'] = pltr_history.loc[i-1, 'OBV'] - pltr_history.loc[i, 'Volume']
            else:
                pltr_history.loc[i, 'OBV'] = pltr_history.loc[i-1, 'OBV']

    pltr_history = pltr_history.iloc[46:,:].reset_index(drop=True)
    
    
    pltr_history = pltr_history[['Date','YearMonDay','Open','High','Low','Volume','Dividends','Stock Splits','Close-t','log_ret','log_ret-t','ACD','MACD','CHO','Highest_T','Lowest_T','S-K','S-D','VPT','W-R','RSI','MOME','AC','PROC','VROC','OBV','Close']]
    
    pltr_history.to_pickle(f"C:\\Users\\Karthik\\Desktop\\Dissertation\\Final_dfs\\{tick}_technical_data_df.pkl")    

In [6]:
pltr_history
pltr_history.shape

(291, 27)